In [1]:
import gc
import warnings

import scipy as sp
import numpy as np
import pandas as pd
import pickle

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm


import sys
sys.path.append("..")
from src import *

warnings.filterwarnings('ignore')

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
class CFG:
    months_agg_list = [2, 6]

    num_agg_func_list = ['last', 'first', 'mean', 'std', 'min', 'max']
    cat_agg_func_list = ['last', 'first', 'median']
    
    drop_uniform = False

In [3]:
features = AmexFeatures()

## TRAIN 

In [4]:
%%time
num_features, cat_features = features.num_origin_features, features.cat_origin_features

train = pd.read_parquet("../input/train_data.parquet")
train = transform_original_cat_features(train, cat_features)

if CFG.drop_uniform:
    num_features = list(set(num_features) - set(features.origin_groups["uniform"]))
    train.drop(features.origin_groups["uniform"], axis=1, inplace=True)

train

CPU times: user 8.96 s, sys: 12.7 s, total: 21.7 s
Wall time: 29.2 s


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,NaN,NaN,0.000630,0.080986,0.708906,0.170600,0.006204,0.358587,0.525351,0.255736,NaN,0.063902,0.059416,0.006466,0.148698,1.335856,0.008207,0.001423,0.207334,0.736463,0.096219,NaN,0.023381,0.002768,0.008322,1.001519,0.008298,0.161345,0.148266,0.922998,0.354596,0.152025,0.118075,0.001882,0.158612,0.065728,0.018385,0.063646,0.199617,0.308233,0.016361,0.401619,0.091071,-10.0,-10.0,0.007126,0.007665,NaN,0.652984,0.008520,-2.0,0.004730,6.0,0.272008,0.008363,0.515222,0.002644,0.009013,0.004808,0.008342,0.119403,0.004802,0.108271,0.050882,NaN,0.007554,0.080422,0.069067,NaN,0.004327,0.007562,NaN,0.007729,0.000272,0.001576,0.004239,0.001434,NaN,0.002271,0.004061,0.007121,0.002456,0.002310,0.003532,0.506612,0.008033,1.009825,0.084683,0.003820,0.007043,0.000438,0.006452,0.000830,0.005055,NaN,0.0,0.005720,0.007084,NaN,0.000198,0.008907,NaN,1.0,0.002537,0.005177,0.006626,0.009705,0.007782,0.002450,1.001101,0.002665,0.007479,0.006893,1.503673,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.894090,0.135561,0.911191,0.974539,0.001243,0.766688,1.008691,1.004587,0.893734,NaN,0.670041,0.009968,0.004572,NaN,1.008949,2.0,NaN,0.004326,NaN,NaN,NaN,1.007336,0.210060,0.676922,0.007871,1.0,0.238250,0.0,4.0,0.232120,0.236266,0.0,0.702280,0.434345,0.003057,0.686516,0.008740,1.0,1.003319,1.007819,1.000080,0.006805,NaN,0.002052,0.005972,NaN,0.004345,0.001535,NaN,NaN,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,NaN,NaN,0.002526,0.069419,0.712795,0.113239,0.006206,0.353630,0.521311,0.223329,NaN,0.065261,0.057744,0.001614,0.149723,1.339794,0.008373,0.001984,0.202778,0.720886,0.099804,NaN,0.030599,0.002749,0.002482,1.009033,0.005136,0.140951,0.143530,0.919414,0.326757,0.156201,0.118737,0.001610,0.148459,0.093935,0.013035,0.065501,0.151387,0.265026,0.017688,0.406326,0.086805,-10.0,-10.0,0.002413,0.007148,NaN,0.647093,0.002238,-2.0,0.003879,6.0,0.188970,0.004030,0.509048,0.004193,0.007842,0.001283,0.006524,0.140611,0.000094,0.101018,0.040469,NaN,0.004832,0.081413,0.074166,NaN,0.004203,0.005304,NaN,0.001864,0.000979,0.009896,0.007597,0.000509,NaN,0.009810,0.000127,0.005966,0.000395,0.001327,0.007773,0.500855,0.000760,1.009461,0.081843,0.000347,0.007789,0.004311,0.002332,0.009469,0.003753,NaN,0.0,0.007584,0.006677,NaN,0.001142,0.005907,NaN,1.0,0.008427,0.008979,0.001854,0.009924,0.005987,0.002247,1.006779,0.002508,0.006827,0.002837,1.503577,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.902135,0.136333,0.919876,0.975625,0.004561,0.786007,1.000084,1.004118,0.906841,NaN,0.668647,0.003921,0.004654,NaN,1.003205,2.0,NaN,0.008707,NaN,NaN,NaN,1.007653,0.184093,0.822281,0.003444,1.0,0.247217,0.0,4.0,0.243532,0.241885,0.0,0.707017,0.430501,0.001306,0.686414,0.000755,1.0,1.008394,1.004333,1.008344,0.004407,NaN,0.00

In [5]:
train["S_2"].min(), train["S_2"].max()

('2017-03-01', '2018-03-31')

In [6]:
%%time
train_agg = preprocessing(train, 
                          num_features, cat_features, 
                          months_agg_list=CFG.months_agg_list, 
                          num_agg_func_list=CFG.num_agg_func_list,
                          cat_agg_func_list=CFG.cat_agg_func_list,
                          is_test=False)
train_agg

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 52 s, sys: 11.8 s, total: 1min 3s
Wall time: 1min 3s


,customer_ID,D_50_last_2m,D_50_first_2m,D_50_mean_2m,D_50_std_2m,D_50_min_2m,D_50_max_2m,R_17_last_2m,R_17_first_2m,R_17_mean_2m,R_17_std_2m,R_17_min_2m,R_17_max_2m,B_39_last_2m,B_39_first_2m,B_39_mean_2m,B_39_std_2m,B_39_min_2m,B_39_max_2m,S_26_last_2m,S_26_first_2m,S_26_mean_2m,S_26_std_2m,S_26_min_2m,S_26_max_2m,D_39_last_2m,D_39_first_2m,D_39_mean_2m,D_39_std_2m,D_39_min_2m,D_39_max_2m,D_75_last_2m,D_75_first_2m,D_75_mean_2m,D_75_std_2m,D_75_min_2m,D_75_max_2m,R_10_last_2m,R_10_first_2m,R_10_mean_2m,R_10_std_2m,R_10_min_2m,R_10_max_2m,R_27_last_2m,R_27_first_2m,R_27_mean_2m,R_27_std_2m,R_27_min_2m,R_27_max_2m,B_17_last_2m,B_17_first_2m,B_17_mean_2m,B_17_std_2m,B_17_min_2m,B_17_max_2m,R_18_last_2m,R_18_first_2m,R_18_mean_2m,R_18_std_2m,R_18_min_2m,R_18_max_2m,R_24_last_2m,R_24_first_2m,R_24_mean_2m,R_24_std_2m,R_24_min_2m,R_24_max_2m,S_6_last_2m,S_6_first_2m,S_6_mean_2m,S_6_std_2m,S_6_min_2m,S_6_max_2m,D_104_last_2m,D_104_first_2m,D_104_mean_2m,D_104_std_2m,D_104_min_2m,D_104_max_2m,D_141_last_2m,D_141_first_2m,D_141_mean_2m,D_141_std_2m,D_141_min_2m,D_141_max_2m,B_11_last_2m,B_11_first_2m,B_11_mean_2m,B_11_std_2m,B_11_min_2m,B_11_max_2m,R_23_last_2m,R_23_first_2m,R_23_mean_2m,R_23_std_2m,R_23_min_2m,R_23_max_2m,B_5_last_2m,B_5_first_2m,B_5_mean_2m,B_5_std_2m,B_5_min_2m,B_5_max_2m,B_19_last_2m,B_19_first_2m,B_19_mean_2m,B_19_std_2m,B_19_min_2m,B_19_max_2m,D_53_last_2m,D_53_first_2m,D_53_mean_2m,D_53_std_2m,D_53_min_2m,D_53_max_2m,D_88_last_2m,D_88_first_2m,D_88_mean_2m,D_88_std_2m,D_88_min_2m,D_88_max_2m,R_2_last_2m,R_2_first_2m,R_2_mean_2m,R_2_std_2m,R_2_min_2m,R_2_max_2m,R_6_last_2m,R_6_first_2m,R_6_mean_2m,R_6_std_2m,R_6_min_2m,R_6_max_2m,D_111_last_2m,D_111_first_2m,D_111_mean_2m,D_111_std_2m,D_111_min_2m,D_111_max_2m,D_110_last_2m,D_110_first_2m,D_110_mean_2m,D_110_std_2m,D_110_min_2m,D_110_max_2m,D_72_last_2m,D_72_first_2m,D_72_mean_2m,D_72_std_2m,D_72_min_2m,D_72_max_2m,B_41_last_2m,B_41_first_2m,B_41_mean_2m,B_41_std_2m,B_41_min_2m,B_41_max_2m,B_21_last_2m,B_21_first_2m,B_21_mean_2m,B_21_std_2m,B_21_min_2m,B_21_max_2m,R_13_last_2m,R_13_first_2m,R_13_mean_2m,R_13_std_2m,R_13_min_2m,R_13_max_2m,D_140_last_2m,D_140_first_2m,D_140_mean_2m,D_140_std_2m,D_140_min_2m,D_140_max_2m,D_61_last_2m,D_61_first_2m,D_61_mean_2m,D_61_std_2m,D_61_min_2m,D_61_max_2m,R_20_last_2m,R_20_first_2m,R_20_mean_2m,R_20_std_2m,R_20_min_2m,R_20_max_2m,R_21_last_2m,R_21_first_2m,R_21_mean_2m,R_21_std_2m,R_21_min_2m,R_21_max_2m,D_54_last_2m,D_54_first_2m,D_54_mean_2m,D_54_std_2m,D_54_min_2m,D_54_max_2m,D_48_last_2m,D_48_first_2m,D_48_mean_2m,D_48_std_2m,D_48_min_2m,D_48_max_2m,D_131_last_2m,D_131_first_2m,D_131_mean_2m,D_131_std_2m,D_131_min_2m,D_131_max_2m,B_9_last_2m,B_9_first_2m,B_9_mean_2m,B_9_std_2m,B_9_min_2m,B_9_max_2m,S_19_last_2m,S_19_first_2m,S_19_mean_2m,S_19_std_2m,S_19_min_2m,S_19_max_2m,B_40_last_2m,B_40_first_2m,B_40_mean_2m,B_40_std_2m,B_40_min_2m,B_40_max_2m,S_5_last_2m,S_5_first_2m,S_5_mean_2m,S_5_std_2m,S_5_min_2m,S_5_max_2m,D_142_last_2m,D_142_first_2m,D_142_mean_2m,D_142_std_2m,D_142_min_2m,D_142_max_2m,S_15_last_2m,S_15_first_2m,S_15_mean_2m,S_15_std_2m,S_15_min_2m,S_15_max_2m,D_60_last_2m,D_60_first_2m,D_60_mean_2m,...,D_96_last_std_diff_6m,D_96_last_min_diff_6m,D_96_last_max_diff_6m,R_3_last_first_diff_2m,R_3_last_mean_diff_2m,R_3_last_std_diff_2m,R_3_last_min_diff_2m,R_3_last_max_diff_2m,R_3_last_first_diff_6m,R_3_last_mean_diff_6m,R_3_last_std_diff_6m,R_3_last_min_diff_6m,R_3_last_max_diff_6m,D_51_last_first_diff_2m,D_51_last_mean_diff_2m,D_51_last_std_diff_2m,D_51_last_min_diff_2m,D_51_last_max_diff_2m,D_51_last_first_diff_6m,D_51_last_mean_diff_6m,D_51_last_std_diff_6m,D_51_last_min_diff_6m,D_51_last_max_diff_6m,B_31_last_first_diff_2m,B_31_last_mean_diff_2m,B_31_last_std_diff_2m,B_31_last_min_diff_2m,B_31_last_max_diff_2m,B_31_last_first_diff_6m,B_31_last_mean_diff_6m,B_31_last_std_diff_6m,B_31_last_min_diff_6m,B_31_last_max_diff_6m,P_3_last_first_diff_2m,P_3_last_mean_diff_2m,P_3_last_std_diff_2m,P_3_last_min_diff_2m,

In [7]:
%%time
train_agg.to_parquet("../tmp/train_agg.parquet")

CPU times: user 59.5 s, sys: 4.14 s, total: 1min 3s
Wall time: 1min 4s


In [8]:
del train, train_agg
gc.collect()

0

## TEST 

In [4]:
%%time
num_features, cat_features = features.num_origin_features, features.cat_origin_features

test = pd.read_parquet("../input/test_data.parquet")
test = transform_original_cat_features(test, cat_features)

if CFG.drop_uniform:
    num_features = list(set(num_features) - set(features.origin_groups["uniform"]))
    test.drop(features.origin_groups["uniform"], axis=1, inplace=True)
    
test

CPU times: user 18 s, sys: 26 s, total: 44 s
Wall time: 56.7 s


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.631315,0.001912,0.010728,0.814497,0.007547,0.168651,0.009971,0.002347,0.113189,NaN,0.007584,0.116402,0.004483,0.009517,0.005178,NaN,0.474383,0.626467,NaN,0.174591,1.250677,1.003925,NaN,0.008718,0.001013,0.103590,0.119043,NaN,-0.002919,NaN,0.002771,0.004416,1.006875,1.005069,0.002810,0.105587,0.119198,0.321676,0.114564,NaN,NaN,0.003420,0.205898,NaN,0.010182,NaN,0.027472,0.588734,0.010489,1.881856,0.051811,-10.0,-10.0,0.008294,0.008663,NaN,0.592103,0.009144,-2.0,0.004986,-2.0,0.190516,0.002675,0.409411,0.009990,NaN,0.005210,NaN,0.014684,0.004310,0.400602,1.134792,0.178862,0.959531,0.217694,0.204590,NaN,0.009799,0.007748,0.002916,0.006896,0.001656,0.004356,NaN,0.002008,NaN,0.002531,0.000348,0.001173,0.004170,0.004585,0.002291,0.508240,0.005093,1.002944,0.133804,0.004322,NaN,0.008857,0.004144,0.009563,0.006942,NaN,0.0,0.003684,0.009909,NaN,0.008908,0.002955,NaN,1.0,0.008999,0.003418,0.002200,0.003781,0.002948,0.004635,1.003810,0.006653,0.007474,0.000192,NaN,0.008761,0.000835,0.004389,0.004115,0.008442,0.009575,0.981078,0.134916,0.954664,0.971086,0.006562,0.002419,NaN,NaN,NaN,NaN,NaN,0.008146,0.007970,NaN,NaN,1.0,NaN,0.005124,NaN,NaN,NaN,1.004130,0.274203,0.004302,NaN,-2.0,NaN,-2.0,-2.0,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,0.0,0.006201,NaN,NaN,0.009664,NaN,NaN,NaN,NaN,0.007082,0.005957,NaN,NaN,NaN,NaN,NaN,NaN,0.004669,NaN,NaN,NaN,0.008281,NaN
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.587042,0.005275,0.011026,0.810848,0.001817,0.241389,0.000166,0.009132,0.123035,NaN,0.006646,0.134878,0.009497,0.026313,0.009029,NaN,0.473587,0.611682,NaN,0.177115,0.234169,1.009301,NaN,0.003146,0.005533,0.101239,0.120991,NaN,0.031222,NaN,0.006497,0.010003,1.005728,1.000432,0.000525,0.160562,0.121490,0.170728,0.184116,NaN,0.028347,0.007156,0.403270,0.014491,0.025130,NaN,0.054790,0.622886,0.021704,2.043278,0.057567,-10.0,-10.0,0.002011,0.000835,NaN,0.590922,0.000165,-2.0,0.001557,-2.0,0.190172,0.001771,0.561424,0.008125,NaN,0.008394,0.000606,0.011945,0.002283,0.605887,0.208381,0.179502,0.961667,0.216526,0.206570,NaN,0.004549,0.003098,NaN,0.003682,0.005516,0.001506,0.001013,0.006444,NaN,0.007169,0.003077,0.005880,0.004056,0.009634,0.001225,0.508681,0.007319,1.003831,0.144626,0.005300,NaN,0.001477,0.002083,0.002234,0.005523,NaN,0.0,0.006410,0.007458,NaN,0.000843,0.007457,NaN,1.0,0.005120,0.002382,0.006557,0.001691,0.004449,0.000574,1.004629,0.004394,0.001169,0.006693,NaN,0.004318,0.008072,0.007888,0.007656,0.006347,0.006952,0.966995,0.138531,0.957019,0.975003,0.004611,0.006549,0.003031,0.008755,NaN,NaN,0.004698,0.000078,0.015463,NaN,1.007666,1.0,NaN,0.000610,NaN,NaN,NaN,1.004495,0.353639,0.276623,NaN,-2.0,NaN,-2.0,-2.0,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,0.0,0.003833,0.008782,0.005579,0.007441,NaN,0.007598,0.008074,NaN,0.007188,0.001253,NaN,NaN,NaN,NaN,NaN,0.000142,0.004940,0.009021,NaN,0.003695,0.003753,0.001460
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-04-25,0.609056,0.003326,0.016390,1.004620,0.000114,0.

In [5]:
cust_date = test[
    (
        ~test[test.columns[1:]].isna()
    ).any(axis=1)
].groupby("customer_ID")["S_2"].max()
cust_date.value_counts()

2019-04-20    30271
2019-04-27    27681
2019-10-26    24035
2019-10-20    22963
2019-10-12    22295
2019-10-04    22179
2019-04-14    21407
2019-04-24    20923
2019-10-16    18910
2019-04-05    18192
2019-04-29    17872
2019-04-15    17769
2019-04-09    17460
2019-04-06    17171
2019-04-12    17085
2019-04-16    16934
2019-04-18    16889
2019-04-17    16736
2019-10-10    16723
2019-10-30    16666
2019-10-28    16456
2019-04-13    16450
2019-10-21    16447
2019-04-26    16272
2019-10-08    16228
2019-04-08    16151
2019-04-22    15806
2019-10-14    15763
2019-10-19    15760
2019-04-10    15625
2019-04-03    15370
2019-10-09    15309
2019-10-11    14934
2019-10-23    14546
2019-10-17    14336
2019-10-29    14177
2019-10-25    14070
2019-10-22    13895
2019-10-24    13762
2019-10-15    13340
2019-04-01    12886
2019-10-01    12479
2019-10-18    11821
2019-04-30    11274
2019-04-02    11094
2019-04-19    10783
2019-04-25    10781
2019-04-04    10691
2019-10-02    10583
2019-04-11    10551


In [6]:
cust_date.str.split("-").str[:2].str.join("-").value_counts()

2019-04    467966
2019-10    456655
Name: S_2, dtype: int64

In [7]:
cust1 = cust_date[cust_date <= '2019-04-31'].index.to_list()
cust2 = list(set(cust_date.index.to_list()) - set(cust1))
test["customer_ID"].unique().shape[0] == len(cust1) + len(cust2)

True

In [8]:
%%time
test_agg1 = preprocessing(test[test["customer_ID"].isin(cust1)], 
                         num_features, cat_features, 
                         months_agg_list=CFG.months_agg_list, 
                         num_agg_func_list=CFG.num_agg_func_list,
                         cat_agg_func_list=CFG.cat_agg_func_list,
                         is_test=True)
test_agg1

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 51.7 s, sys: 9.12 s, total: 1min
Wall time: 1min


,customer_ID,R_19_last_2m,R_19_first_2m,R_19_mean_2m,R_19_std_2m,R_19_min_2m,R_19_max_2m,D_121_last_2m,D_121_first_2m,D_121_mean_2m,D_121_std_2m,D_121_min_2m,D_121_max_2m,D_70_last_2m,D_70_first_2m,D_70_mean_2m,D_70_std_2m,D_70_min_2m,D_70_max_2m,S_26_last_2m,S_26_first_2m,S_26_mean_2m,S_26_std_2m,S_26_min_2m,S_26_max_2m,D_140_last_2m,D_140_first_2m,D_140_mean_2m,D_140_std_2m,D_140_min_2m,D_140_max_2m,D_58_last_2m,D_58_first_2m,D_58_mean_2m,D_58_std_2m,D_58_min_2m,D_58_max_2m,D_60_last_2m,D_60_first_2m,D_60_mean_2m,D_60_std_2m,D_60_min_2m,D_60_max_2m,P_2_last_2m,P_2_first_2m,P_2_mean_2m,P_2_std_2m,P_2_min_2m,P_2_max_2m,B_10_last_2m,B_10_first_2m,B_10_mean_2m,B_10_std_2m,B_10_min_2m,B_10_max_2m,B_16_last_2m,B_16_first_2m,B_16_mean_2m,B_16_std_2m,B_16_min_2m,B_16_max_2m,D_107_last_2m,D_107_first_2m,D_107_mean_2m,D_107_std_2m,D_107_min_2m,D_107_max_2m,D_108_last_2m,D_108_first_2m,D_108_mean_2m,D_108_std_2m,D_108_min_2m,D_108_max_2m,B_3_last_2m,B_3_first_2m,B_3_mean_2m,B_3_std_2m,B_3_min_2m,B_3_max_2m,D_119_last_2m,D_119_first_2m,D_119_mean_2m,D_119_std_2m,D_119_min_2m,D_119_max_2m,D_73_last_2m,D_73_first_2m,D_73_mean_2m,D_73_std_2m,D_73_min_2m,D_73_max_2m,D_122_last_2m,D_122_first_2m,D_122_mean_2m,D_122_std_2m,D_122_min_2m,D_122_max_2m,R_11_last_2m,R_11_first_2m,R_11_mean_2m,R_11_std_2m,R_11_min_2m,R_11_max_2m,B_29_last_2m,B_29_first_2m,B_29_mean_2m,B_29_std_2m,B_29_min_2m,B_29_max_2m,D_44_last_2m,D_44_first_2m,D_44_mean_2m,D_44_std_2m,D_44_min_2m,D_44_max_2m,B_5_last_2m,B_5_first_2m,B_5_mean_2m,B_5_std_2m,B_5_min_2m,B_5_max_2m,B_15_last_2m,B_15_first_2m,B_15_mean_2m,B_15_std_2m,B_15_min_2m,B_15_max_2m,R_16_last_2m,R_16_first_2m,R_16_mean_2m,R_16_std_2m,R_16_min_2m,R_16_max_2m,S_3_last_2m,S_3_first_2m,S_3_mean_2m,S_3_std_2m,S_3_min_2m,S_3_max_2m,B_12_last_2m,B_12_first_2m,B_12_mean_2m,B_12_std_2m,B_12_min_2m,B_12_max_2m,R_21_last_2m,R_21_first_2m,R_21_mean_2m,R_21_std_2m,R_21_min_2m,R_21_max_2m,R_1_last_2m,R_1_first_2m,R_1_mean_2m,R_1_std_2m,R_1_min_2m,R_1_max_2m,D_52_last_2m,D_52_first_2m,D_52_mean_2m,D_52_std_2m,D_52_min_2m,D_52_max_2m,D_54_last_2m,D_54_first_2m,D_54_mean_2m,D_54_std_2m,D_54_min_2m,D_54_max_2m,D_128_last_2m,D_128_first_2m,D_128_mean_2m,D_128_std_2m,D_128_min_2m,D_128_max_2m,R_15_last_2m,R_15_first_2m,R_15_mean_2m,R_15_std_2m,R_15_min_2m,R_15_max_2m,B_7_last_2m,B_7_first_2m,B_7_mean_2m,B_7_std_2m,B_7_min_2m,B_7_max_2m,D_83_last_2m,D_83_first_2m,D_83_mean_2m,D_83_std_2m,D_83_min_2m,D_83_max_2m,R_9_last_2m,R_9_first_2m,R_9_mean_2m,R_9_std_2m,R_9_min_2m,R_9_max_2m,D_136_last_2m,D_136_first_2m,D_136_mean_2m,D_136_std_2m,D_136_min_2m,D_136_max_2m,D_81_last_2m,D_81_first_2m,D_81_mean_2m,D_81_std_2m,D_81_min_2m,D_81_max_2m,S_20_last_2m,S_20_first_2m,S_20_mean_2m,S_20_std_2m,S_20_min_2m,S_20_max_2m,D_143_last_2m,D_143_first_2m,D_143_mean_2m,D_143_std_2m,D_143_min_2m,D_143_max_2m,B_37_last_2m,B_37_first_2m,B_37_mean_2m,B_37_std_2m,B_37_min_2m,B_37_max_2m,S_19_last_2m,S_19_first_2m,S_19_mean_2m,S_19_std_2m,S_19_min_2m,S_19_max_2m,D_93_last_2m,D_93_first_2m,D_93_mean_2m,D_93_std_2m,D_93_min_2m,D_93_max_2m,R_25_last_2m,R_25_first_2m,R_25_mean_2m,R_25_std_2m,R_25_min_2m,R_25_max_2m,D_110_last_2m,D_110_first_2m,D_110_mean_2m,...,B_28_last_mean_diff_6m,B_28_last_std_diff_6m,B_28_last_min_diff_6m,B_28_last_max_diff_6m,S_16_last_first_diff_2m,S_16_last_mean_diff_2m,S_16_last_std_diff_2m,S_16_last_min_diff_2m,S_16_last_max_diff_2m,S_16_last_first_diff_6m,S_16_last_mean_diff_6m,S_16_last_std_diff_6m,S_16_last_min_diff_6m,S_16_last_max_diff_6m,D_69_last_first_diff_2m,D_69_last_mean_diff_2m,D_69_last_std_diff_2m,D_69_last_min_diff_2m,D_69_last_max_diff_2m,D_69_last_first_diff_6m,D_69_last_mean_diff_6m,D_69_last_std_diff_6m,D_69_last_min_diff_6m,D_69_last_max_diff_6m,B_21_last_first_diff_2m,B_21_last_mean_diff_2m,B_21_last_std_diff_2m,B_21_last_min_diff_2m,B_21_last_max_diff_2m,B_21_last_first_diff_6m,B_21_last_mean_diff_6m,B_21_last_std_diff_6m,B_21_last_min_diff_6m,B_21_last_max_diff_6m,S_15_last_first_diff_2m,S_15_last_mean_diff_2

In [9]:
test_agg2 = preprocessing(test[test["customer_ID"].isin(cust2)], 
                         num_features, cat_features, 
                         months_agg_list=CFG.months_agg_list, 
                         num_agg_func_list=CFG.num_agg_func_list,
                         cat_agg_func_list=CFG.cat_agg_func_list,
                         is_test=True)
test_agg2

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

,customer_ID,R_19_last_2m,R_19_first_2m,R_19_mean_2m,R_19_std_2m,R_19_min_2m,R_19_max_2m,D_121_last_2m,D_121_first_2m,D_121_mean_2m,D_121_std_2m,D_121_min_2m,D_121_max_2m,D_70_last_2m,D_70_first_2m,D_70_mean_2m,D_70_std_2m,D_70_min_2m,D_70_max_2m,S_26_last_2m,S_26_first_2m,S_26_mean_2m,S_26_std_2m,S_26_min_2m,S_26_max_2m,D_140_last_2m,D_140_first_2m,D_140_mean_2m,D_140_std_2m,D_140_min_2m,D_140_max_2m,D_58_last_2m,D_58_first_2m,D_58_mean_2m,D_58_std_2m,D_58_min_2m,D_58_max_2m,D_60_last_2m,D_60_first_2m,D_60_mean_2m,D_60_std_2m,D_60_min_2m,D_60_max_2m,P_2_last_2m,P_2_first_2m,P_2_mean_2m,P_2_std_2m,P_2_min_2m,P_2_max_2m,B_10_last_2m,B_10_first_2m,B_10_mean_2m,B_10_std_2m,B_10_min_2m,B_10_max_2m,B_16_last_2m,B_16_first_2m,B_16_mean_2m,B_16_std_2m,B_16_min_2m,B_16_max_2m,D_107_last_2m,D_107_first_2m,D_107_mean_2m,D_107_std_2m,D_107_min_2m,D_107_max_2m,D_108_last_2m,D_108_first_2m,D_108_mean_2m,D_108_std_2m,D_108_min_2m,D_108_max_2m,B_3_last_2m,B_3_first_2m,B_3_mean_2m,B_3_std_2m,B_3_min_2m,B_3_max_2m,D_119_last_2m,D_119_first_2m,D_119_mean_2m,D_119_std_2m,D_119_min_2m,D_119_max_2m,D_73_last_2m,D_73_first_2m,D_73_mean_2m,D_73_std_2m,D_73_min_2m,D_73_max_2m,D_122_last_2m,D_122_first_2m,D_122_mean_2m,D_122_std_2m,D_122_min_2m,D_122_max_2m,R_11_last_2m,R_11_first_2m,R_11_mean_2m,R_11_std_2m,R_11_min_2m,R_11_max_2m,B_29_last_2m,B_29_first_2m,B_29_mean_2m,B_29_std_2m,B_29_min_2m,B_29_max_2m,D_44_last_2m,D_44_first_2m,D_44_mean_2m,D_44_std_2m,D_44_min_2m,D_44_max_2m,B_5_last_2m,B_5_first_2m,B_5_mean_2m,B_5_std_2m,B_5_min_2m,B_5_max_2m,B_15_last_2m,B_15_first_2m,B_15_mean_2m,B_15_std_2m,B_15_min_2m,B_15_max_2m,R_16_last_2m,R_16_first_2m,R_16_mean_2m,R_16_std_2m,R_16_min_2m,R_16_max_2m,S_3_last_2m,S_3_first_2m,S_3_mean_2m,S_3_std_2m,S_3_min_2m,S_3_max_2m,B_12_last_2m,B_12_first_2m,B_12_mean_2m,B_12_std_2m,B_12_min_2m,B_12_max_2m,R_21_last_2m,R_21_first_2m,R_21_mean_2m,R_21_std_2m,R_21_min_2m,R_21_max_2m,R_1_last_2m,R_1_first_2m,R_1_mean_2m,R_1_std_2m,R_1_min_2m,R_1_max_2m,D_52_last_2m,D_52_first_2m,D_52_mean_2m,D_52_std_2m,D_52_min_2m,D_52_max_2m,D_54_last_2m,D_54_first_2m,D_54_mean_2m,D_54_std_2m,D_54_min_2m,D_54_max_2m,D_128_last_2m,D_128_first_2m,D_128_mean_2m,D_128_std_2m,D_128_min_2m,D_128_max_2m,R_15_last_2m,R_15_first_2m,R_15_mean_2m,R_15_std_2m,R_15_min_2m,R_15_max_2m,B_7_last_2m,B_7_first_2m,B_7_mean_2m,B_7_std_2m,B_7_min_2m,B_7_max_2m,D_83_last_2m,D_83_first_2m,D_83_mean_2m,D_83_std_2m,D_83_min_2m,D_83_max_2m,R_9_last_2m,R_9_first_2m,R_9_mean_2m,R_9_std_2m,R_9_min_2m,R_9_max_2m,D_136_last_2m,D_136_first_2m,D_136_mean_2m,D_136_std_2m,D_136_min_2m,D_136_max_2m,D_81_last_2m,D_81_first_2m,D_81_mean_2m,D_81_std_2m,D_81_min_2m,D_81_max_2m,S_20_last_2m,S_20_first_2m,S_20_mean_2m,S_20_std_2m,S_20_min_2m,S_20_max_2m,D_143_last_2m,D_143_first_2m,D_143_mean_2m,D_143_std_2m,D_143_min_2m,D_143_max_2m,B_37_last_2m,B_37_first_2m,B_37_mean_2m,B_37_std_2m,B_37_min_2m,B_37_max_2m,S_19_last_2m,S_19_first_2m,S_19_mean_2m,S_19_std_2m,S_19_min_2m,S_19_max_2m,D_93_last_2m,D_93_first_2m,D_93_mean_2m,D_93_std_2m,D_93_min_2m,D_93_max_2m,R_25_last_2m,R_25_first_2m,R_25_mean_2m,R_25_std_2m,R_25_min_2m,R_25_max_2m,D_110_last_2m,D_110_first_2m,D_110_mean_2m,...,B_28_last_mean_diff_6m,B_28_last_std_diff_6m,B_28_last_min_diff_6m,B_28_last_max_diff_6m,S_16_last_first_diff_2m,S_16_last_mean_diff_2m,S_16_last_std_diff_2m,S_16_last_min_diff_2m,S_16_last_max_diff_2m,S_16_last_first_diff_6m,S_16_last_mean_diff_6m,S_16_last_std_diff_6m,S_16_last_min_diff_6m,S_16_last_max_diff_6m,D_69_last_first_diff_2m,D_69_last_mean_diff_2m,D_69_last_std_diff_2m,D_69_last_min_diff_2m,D_69_last_max_diff_2m,D_69_last_first_diff_6m,D_69_last_mean_diff_6m,D_69_last_std_diff_6m,D_69_last_min_diff_6m,D_69_last_max_diff_6m,B_21_last_first_diff_2m,B_21_last_mean_diff_2m,B_21_last_std_diff_2m,B_21_last_min_diff_2m,B_21_last_max_diff_2m,B_21_last_first_diff_6m,B_21_last_mean_diff_6m,B_21_last_std_diff_6m,B_21_last_min_diff_6m,B_21_last_max_diff_6m,S_15_last_first_diff_2m,S_15_last_mean_diff_2

### Merge 

In [10]:
# test_agg1 = pd.read_parquet("../tmp/test_agg1.parquet")
# test_agg2 = pd.read_parquet("../tmp/test_agg2.parquet")

test_agg = pd.concat([test_agg1, test_agg2])

del test_agg1, test_agg2
gc.collect()

test_agg.to_parquet("../tmp/test_agg.parquet")

del test_agg
gc.collect()

0